In [ ]:
# !pip install simhash

# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404346,404346,789792,789793,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404347,404347,789794,789795,Do you believe there is life after death?,Is it true that there is life after death?,1
404348,404348,789796,789797,What is one coin?,What's this coin?,0
404349,404349,789798,789799,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [ ]:
# minhash function calculates similarity measure using minhash metrics
def minhash(input_question, compare_question):
    score = 0.0
    try:
      shingles = lambda s: set(s[i:i+3] for i in range(len(s)-2))
    except TypeError:
      pass
    try: 
      jaccard_distance = lambda seta, setb: len(seta & setb)/float(len(seta | setb))
    except:
      pass
    try:
        score = jaccard_distance(shingles(input_question), shingles(compare_question))
    except ZeroDivisionError:
        print('ZeroDivisionError')

    return score

#Source: betterprogramming.pub/identify-similarities-between-sentences-in-python-e9f71d454d1d

In [ ]:
print(type(Quora_data))

In [ ]:
print(type(Quora_data))

Quora_data = Quora_data[:-1]

index= 0
scores = [0]*(len(Quora_data))
for index, row in Quora_data.iterrows():
  # print('Q1',row['question1'])
  # print('Q2',row['question2'])
  # if (not math.isnan(float(row['question1']))) & (not math.isnan(float(row['question2']))):
  # print(index)
  if((row['question1'] != 'nan') and (row['question2'] != 'nan')):
    scores[index] = (minhash(str(row['question1']), str(row['question2'])))
    # index +1
  # break
  # if index >100:
Quora_data['Jaccard_score'] = scores
  #   break
  # else:
  #   index+=1

In [ ]:
Quora_data.head(20)

,id,qid1,qid2,question1,question2,is_duplicate,Jaccard_score
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.872727
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.322917
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.288660
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.172043
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,0.336134
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,0.000000
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,0.294118
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,0.600000
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,0.285714


LSH using Word2vec


In [ ]:
from gensim.models import KeyedVectors
import json
from simhash import Simhash, SimhashIndex
import numpy as np

In [ ]:
model_path = "/content/drive/MyDrive/5430 Project/QQ's (Quora Questions)/"
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_w2v_AP    = load_wordvec_model('Word2Vec Google News', 'GoogleNews-vectors-negative300.bin.gz', True)

Loading Word2Vec Google News model...
Finished loading Word2Vec Google News model...


In [ ]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [ ]:
# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [ ]:
index= 0
scores = [0]*(len(Quora_data))
for index, row in Quora_data.iterrows():
  if(len(str(row['question1']))>0 and len(str(row['question1']))>0):
    try:
      scores[index] = (calc_similarity(str(row['question1']), str(row['question2']), model_w2v_AP))
    except ZeroDivisionError:
      pass

Quora_data['Simhash_score'] = scores

In [ ]:
Quora_data.head(20)

,id,qid1,qid2,question1,question2,is_duplicate,Jaccard_score,Simhash_score
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.872727,0.973258
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.322917,0.673686
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.288660,0.775142
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.455245
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.172043,0.587267
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,0.336134,0.764968
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,0.000000,0.380040
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,0.294118,0.907624
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,0.600000,1.000000
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,0.285714,0.809821


Cosine Similarity

In [ ]:
#https://stackoverflow.com/questions/15173225/calculate-cosine-similarity-given-2-sentence-strings

import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


In [ ]:
index= 0
scores = [0]*(len(Quora_data))
for index, row in Quora_data.iterrows():
  # print(row['question1'], row['question2'])
  scores[index] = (get_cosine(text_to_vector(str(row['question1'])), text_to_vector(str(row['question2']))))
  # break
  # if index >100:
Quora_data['Cosine_score'] = scores
  #   break
  # else:
  #   index+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
Quora_data.head(20)

,id,qid1,qid2,question1,question2,is_duplicate,Jaccard_score,Simhash_score,Cosine_score
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.872727,0.973258,0.944911
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.322917,0.673686,0.536875
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.288660,0.775142,0.253546
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.455245,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.172043,0.587267,0.419314
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,0.336134,0.764968,0.556415
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,0.000000,0.380040,0.000000
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,0.294118,0.907624,0.503953
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,0.600000,1.000000,0.801784
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,0.285714,0.809821,0.444444


In [ ]:
#DICE(sørensen sumting)
# https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Dice%27s_coefficient

def dice_coefficient(a, b):
    """dice coefficient 2nt/(na + nb)."""
    if not len(a) or not len(b): return 0.0
    if len(a) == 1:  a=a+u'.'
    if len(b) == 1:  b=b+u'.'
    
    a_bigram_list=[]
    for i in range(len(a)-1):
      a_bigram_list.append(a[i:i+2])
    b_bigram_list=[]
    for i in range(len(b)-1):
      b_bigram_list.append(b[i:i+2])
      
    a_bigrams = set(a_bigram_list)
    b_bigrams = set(b_bigram_list)
    overlap = len(a_bigrams & b_bigrams)
    dice_coeff = overlap * 2.0/(len(a_bigrams) + len(b_bigrams))
    return dice_coeff

In [ ]:
index= 0
scores = [0]*(len(Quora_data))
for index, row in Quora_data.iterrows():
  # print(row['question1'], row['question2'])
  scores[index] = (dice_coefficient((str(row['question1'])), (str(row['question2']))))
  # break
  # if index >100:
Quora_data['Dice_score'] = scores
  #   break
  # else:
  #   index+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
Quora_data.head(20)

,id,qid1,qid2,question1,question2,is_duplicate,Jaccard_score,Simhash_score,Cosine_score,Dice_score
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.872727,0.973258,0.944911,0.939759
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.322917,0.673686,0.536875,0.626087
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.288660,0.775142,0.253546,0.598291
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.455245,0.000000,0.127660
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.172043,0.587267,0.419314,0.448980
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,0.336134,0.764968,0.556415,0.647059
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,0.000000,0.380040,0.000000,0.112676
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,0.294118,0.907624,0.503953,0.562500
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,0.600000,1.000000,0.801784,0.800000
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,0.285714,0.809821,0.444444,0.553191


In [ ]:
#Jensen-Shannon distance

Evaluation
<!--  -->

In [ ]:

def assignResults(Quora_data,Jaccard_Threshold,Simhash_Threshold,Cosine_Threshold, Dice_Threshold):
  result_J = [0]*(len(Quora_data))
  result_S = [0]*(len(Quora_data))
  result_C = [0]*(len(Quora_data))
  result_D = [0]*(len(Quora_data))


  for index, row in Quora_data.iterrows():
    if(row['Jaccard_score'] > Jaccard_Threshold):
      result_J[index] = 1
    else:
      result_J[index] = 0
    if(row['Simhash_score'] > Simhash_Threshold):
      result_S[index] = 1
    else:
      result_S[index] = 0
    if(row['Cosine_score'] > Cosine_Threshold):
      result_C[index] = 1
    else:
      result_C[index] = 0
    if(row['Dice_score'] > Dice_Threshold):
      result_D[index] = 1
    else:
      result_D[index] = 0

  Quora_data['Jaccard_Result'] = result_J
  Quora_data['Simhash_Result'] = result_S
  Quora_data['Cosine_Result'] = result_C
  Quora_data['Dice_Result'] = result_D


  return Quora_data


    

In [ ]:
Jaccard_Threshold = 0.7
Simhash_Threshold = 0.7
Cosine_Threshold = 0.7
Dice_Threshold = 0.7


Quora_data = assignResults(Quora_data,Jaccard_Threshold,Simhash_Threshold,Cosine_Threshold, Dice_Threshold)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
Quora_data.head(20)

,id,qid1,qid2,question1,question2,is_duplicate,Jaccard_score,Simhash_score,Cosine_score,Dice_score,Jaccard_Result,Simhash_Result,Cosine_Result,Dice_Result
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.872727,0.973258,0.944911,0.939759,1,1,1,1
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.322917,0.673686,0.536875,0.626087,0,0,0,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.288660,0.775142,0.253546,0.598291,0,1,0,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.455245,0.000000,0.127660,0,0,0,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.172043,0.587267,0.419314,0.448980,0,0,0,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,0.336134,0.764968,0.556415,0.647059,0,1,0,0
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,0.000000,0.380040,0.000000,0.112676,0,0,0,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,0.294118,0.907624,0.503953,0.562500,0,1,0,0
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,0.600000,1.000000,0.801784,0.800000,0,1,1,1
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,0.285714,0.809821,0.444444,0.553191,0,1,0,0


In [ ]:
def calculate_accuracy(df, column):
  true_positive = 0
  true_negative = 0

  for index, row in df.iterrows():
    if(row['is_duplicate'] == row[column] and row['is_duplicate'] == 1):
      true_positive +=1
    elif(row['is_duplicate'] == row[column] and row['is_duplicate'] == 0):
      true_negative +=1
  
  return (true_positive+true_negative)/(len(df))



In [ ]:
#slower to use my functions. Faster to use Sklearn as implemented below
# print(calculate_accuracy(Quora_data,'Jaccard_Result'))
# print(calculate_accuracy(Quora_data,'Simhash_Result'))
# print(calculate_accuracy(Quora_data,'Cosine_Result'))

In [ ]:
def calculate_recall(df,column):
  true_positive = 0
  false_negative = 0

  for index, row in df.iterrows():
    if(row['is_duplicate'] == row[column] and row['is_duplicate'] == 1):
      true_positive +=1
    elif(row['is_duplicate'] == 1 and row[column] == 0):
      false_negative +=1
  

  return true_positive/(true_positive+false_negative)

In [ ]:
def calculate_precision(df,column):
  true_positive = 0
  false_positive = 0

  for index, row in df.iterrows():
    if(row['is_duplicate'] == row[column] and row['is_duplicate'] == 1):
      true_positive +=1
    elif(row['is_duplicate'] == 0 and row[column] == 1):
      false_negative +=1

  return true_positive/(true_positive+false_negative)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score ## need to check additional inputs to functions

print(accuracy_score(Quora_data['is_duplicate'], Quora_data['Jaccard_Result']))
print(accuracy_score(Quora_data['is_duplicate'], Quora_data['Simhash_Result']))
print(accuracy_score(Quora_data['is_duplicate'], Quora_data['Cosine_Result']))

print(precision_score(Quora_data['is_duplicate'], Quora_data['Jaccard_Result']))
print(precision_score(Quora_data['is_duplicate'], Quora_data['Simhash_Result']))
print(precision_score(Quora_data['is_duplicate'], Quora_data['Cosine_Result']))

print(recall_score(Quora_data['is_duplicate'], Quora_data['Jaccard_Result']))
print(recall_score(Quora_data['is_duplicate'], Quora_data['Simhash_Result']))
print(recall_score(Quora_data['is_duplicate'], Quora_data['Cosine_Result']))

0.6342500309138123
0.5760207740818598
0.6408458019042909
0.518278347585708
0.4628940308517773
0.5217762413844611
0.1343616465513777
0.9245107363401337
0.3275420947584156


Optimization

In [ ]:
#loop through thresholds and check accuracy

Optimization_df = pd.DataFrame(columns=['Threshold','Jaccard Accuracy', 'Simhash Accuracy', 'Cosine Accuracy', 'Dice Accuracy'])

for t in range(50,100,5):
  # Quora_data = assignResults(Quora_data,t,t,t)
  # print(accuracy_score(assignResults(Quora_data,t,t,t)['is_duplicate'], assignResults(Quora_data,t,t,t)['Jaccard_Result']))
  t=t/100
  print(t)
  Optimization_df = Optimization_df.append({'Threshold': t
                        ,'Jaccard Accuracy':accuracy_score(assignResults(Quora_data,t,t,t,t)['is_duplicate'], assignResults(Quora_data,t,t,t,t)['Jaccard_Result'])
                        ,'Simhash Accuracy':accuracy_score(assignResults(Quora_data,t,t,t,t)['is_duplicate'], assignResults(Quora_data,t,t,t,t)['Simhash_Result'])
                        ,'Cosine Accuracy':accuracy_score(assignResults(Quora_data,t,t,t,t)['is_duplicate'], assignResults(Quora_data,t,t,t,t)['Cosine_Result'])
                        ,'Dice Accuracy':accuracy_score(assignResults(Quora_data,t,t,t,t)['is_duplicate'], assignResults(Quora_data,t,t,t,t)['Dice_Result'])},ignore_index=True)
  # break

In [ ]:
Optimization_df

,Threshold,Jaccard Accuracy,Simhash Accuracy,Cosine Accuracy,Dice Accuracy
0,0.50,0.653721,0.423779,0.654700,0.613409
1,0.55,0.648097,0.450664,0.655877,0.651893
2,0.60,0.642990,0.484897,0.652553,0.674280
3,0.65,0.638286,0.527256,0.646811,0.676595
4,0.70,0.634250,0.576021,0.640846,0.670681
5,0.75,0.631540,0.623074,0.633098,0.662035
6,0.80,0.631183,0.657413,0.628228,0.652979
7,0.85,0.634233,0.665879,0.623408,0.645532
8,0.90,0.634129,0.649257,0.623720,0.640074
9,0.95,0.631915,0.626480,0.631065,0.639082
